In [ ]:
import os
import requests

In [ ]:
from bs4 import BeautifulSoup
from settings import (
    CARTOCIUDAD_LIST,
    DATA_FOLDER,
    DOWNLOAD_URL, 
    GVA_HOST,
    REGISTER_FILENAME,
)
from tqdm import tqdm
from zipfile import ZipFile


In [ ]:
# Flags to force GVA data download and CNIG data Unzip
GVA_DOWNLOAD_AGAIN = False
CNIG_UNZIP = False

In [ ]:
LOCAL_REGISTER_FILENAME = os.path.join(DATA_FOLDER,REGISTER_FILENAME)

In [ ]:
download_again = (not os.path.exists(LOCAL_REGISTER_FILENAME)) or (GVA_DOWNLOAD_AGAIN)
if download_again:
    # Get new register file link
    web_response = requests.get(f'{GVA_HOST}/{DOWNLOAD_URL}')

    webcontent = web_response.text
    soup = BeautifulSoup(webcontent, 'html.parser')

    register_url = [ reg.get('href') for reg in soup.select('a') if REGISTER_FILENAME in reg.get('href','')]
    
    # Download data from the new register link
    register_link = f'{GVA_HOST}{register_url[0]}'
    with requests.get(register_link, stream=True) as register_response:
        # Save register data in disk
        with open(LOCAL_REGISTER_FILENAME, 'wb') as lrf:
            for chunk in register_response.iter_content(chunk_size=8192):
                lrf.write(chunk)
        print(f'INFO: Downloaded NEW {LOCAL_REGISTER_FILENAME}')
else:
    print(f'INFO: No NEW Download: Info in {LOCAL_REGISTER_FILENAME}')

In [ ]:
# Process the list of provinces from settings
for provzipname in tqdm(CARTOCIUDAD_LIST):
    zipname =  os.path.join(DATA_FOLDER,'zip', f'CARTOCIUDAD_CALLEJERO_{provzipname}.zip')
    if os.path.exists(zipname):
        with ZipFile(zipname, 'r') as zipfile:
            for filearchive in zipfile.filelist:
                internal_filename = filearchive.filename
                # We only want the PK data
                if 'PK' in internal_filename:    
                    old_name = os.path.join(DATA_FOLDER,'PK',internal_filename)
                    new_name = os.path.join(DATA_FOLDER,'PK', f"{provzipname}_{internal_filename.split('/')[1]}")
                    unzip_again = (not os.path.exists(new_name)) or (CNIG_UNZIP)
                    if unzip_again:
                        zipfile.extract(internal_filename, os.path.join(f'{DATA_FOLDER}','PK'))
                        os.rename(old_name, new_name)
                        os.removedirs(os.path.join(DATA_FOLDER,'PK',f'CARTOCIUDAD_CALLEJERO_{provzipname}/'))
                        print(f'INFO: Unziped {new_name}')
                    else:
                        print(f'INFO: No NEW Unzip for {new_name}')
    else:
        print(f'Download CARTOCiudad files for {provzipname} from CNIG: http://centrodedescargas.cnig.es/')